In [1]:
import pandas as pd
import numpy as np

In [2]:
dt = pd.read_pickle('evolved_exons.pkl')
# please download annotations.total.txt.gz from https://doi.org/10.5281/zenodo.4273072
anno = pd.read_csv('annotations.total.txt', sep='\t', comment='#', header=None, usecols=[0,2,4,11,12])
anno.columns = ['chr','tid','strand','exon_start','exon_end']
anno = pd.merge(dt[['tid','history','ipos','evolved_ipos','nth_intron','seq_length']], anno, on='tid')
anno['exon_start'] = anno.exon_start.str.split(',')
anno['exon_end'] = anno.exon_end.str.split(',')
anno['intron_range'] = anno[['strand','nth_intron','exon_start','exon_end']].values.tolist()

In [3]:
anno['evolved_ipos'] = anno.evolved_ipos.astype(int)
anno['seq_length'] = anno.seq_length.astype(int)
anno['check_ipos'] = anno[['ipos','evolved_ipos']].values.tolist()
anno['check_ipos'] = anno.check_ipos.apply(lambda x: [i for i in x[0] if i==x[1]])
al = anno[anno.history=='loss']
agp = anno[anno.astype(str)['check_ipos'] != '[]']
anno = pd.concat([al,agp])
anno = anno[(anno.evolved_ipos!=0) & (anno.evolved_ipos!=anno.seq_length)]
anno['check_ends'] = anno[['ipos','seq_length']].values.tolist()
anno = anno[(anno.check_ends.apply(lambda x: x[0][0]!=0)) & 
            (anno.check_ends.apply(lambda x: x[0][-1]!=x[1]))]

In [4]:
a = anno[anno.history=='gain'].reset_index(drop=True)
a['intron_range'] = a.intron_range.apply(lambda x: 
                                         [x[3][x[1]-1], x[2][x[1]]] 
                                         if x[0]=='+' else 
                                         [x[3][len(x[2])-x[1]-1], x[2][len(x[2])-x[1]]])

In [5]:
a['intron_start'] = a.intron_range.apply(lambda x: x[0])
a['intron_end'] = a.intron_range.apply(lambda x: x[1])

b = a[['chr','intron_start','intron_end','tid','nth_intron','strand']].drop_duplicates()
b.reset_index(drop=True, inplace=True)
b[['taxid','id']] = b.tid.str.split('|', expand=True)
b['chr'] = b.taxid + '|' + b.chr
b.drop(['taxid','id'], axis=1, inplace=True)
b.to_csv('../repeat_elements/gained_introns.bed', sep='\t', index=None, header=None)

In [7]:
a = anno[anno.history=='present'].reset_index(drop=True)
a['intron_range'] = a.intron_range.apply(lambda x: 
                                         [x[3][x[1]-1], x[2][x[1]]] 
                                         if x[0]=='+' else 
                                         [x[3][len(x[2])-x[1]-1], x[2][len(x[2])-x[1]]])

In [8]:
a['intron_start'] = a.intron_range.apply(lambda x: x[0])
a['intron_end'] = a.intron_range.apply(lambda x: x[1])

b = a[['chr','intron_start','intron_end','tid','nth_intron','strand']].drop_duplicates()
b.reset_index(drop=True, inplace=True)
b[['taxid','id']] = b.tid.str.split('|', expand=True)
b['chr'] = b.taxid + '|' + b.chr
b.drop(['taxid','id'], axis=1, inplace=True)
b.to_csv('../repeat_elements/present_introns.bed', sep='\t', index=None, header=None)